In [1]:
%cd ..

/Users/aydar/Desktop/AI TALENT HUB/DndBattleManager


/Users/aydar/Desktop/AI TALENT HUB/DndBattleManager/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

In [4]:
from agent.model import ChatGPTAPI
from agent.rag import retrieve_related_chunks
import json

#### Модель

In [5]:
import json, pickle


def get_help():
    # Open and read the JSON file
    with open('client/map.json', 'r') as file:
        map_info = json.load(file)

    # Создаем имя файла для кеша на основе промта
    cache_dir = 'cache'
    cache_file = f'cache/{hash(str(map_info))}.pkl'
    
    # Проверяем, существует ли кеш
    if os.path.exists(cache_file):
        with open(cache_file, 'rb') as f:
            help = pickle.load(f)
            print("Используем кешированный ответ.")
            return help
    
    map_desc_help = retrieve_related_chunks(map_info['map']['description'])
    char_desc_help = []
    for character in map_info['characters']:
        char_desc_help.append(retrieve_related_chunks(character['description']))
    help = ''
    for res in map_desc_help['chunks']:
        help += res['description']
    for char in char_desc_help:
        for res in char['chunks']:
            help += res['description']
    
    # Сохраняем ответ в кеш
    os.makedirs(cache_dir, exist_ok=True)
    with open(cache_file, 'ab') as f:
        pickle.dump('Дополнительная информация:\n' + help, f)
        print("Ответ сохранен в кеш.")

    return 'Дополнительная информация:\n' + help

In [6]:
model = ChatGPTAPI()

In [7]:

with open('client/map.json', 'r') as file:
    map_info = json.load(file)
dop_info = get_help()
query = str(map_info) + "\n\n" + dop_info

Ответ сохранен в кеш.


In [8]:
res = model.generate_action(query)
print('Ответ модели:\n', res)

Ответ модели:
 Гоблин2, находящийся в центре сражения, осматривает поле боя и замечает Героя, который находится на расстоянии 3 клеток (примерно 15 футов) от него. Учитывая свою храбрость и желание сразиться с этим могучим волшебником, Гоблин2 решает атаковать.

Гоблин2 использует свою палку (урон 1d4) и подходит на 1 клетку (координаты 2, 6) к Герою, чтобы находиться в пределах досягаемости для атаки. Теперь он совершает атаку.

Для атаки Гоблин2 бросает кубик 1d20 и добавляет свой модификатор атаки (предположим, что это +4). Результат броска составляет 12, что в сумме дает 16. Это больше, чем класс доспеха Героя (10), поэтому атака успешна.

Теперь Гоблин2 бросает кубик 1d4, чтобы определить урон. Результат броска составляет 3. Герой получает 3 урона и теперь его здоровье составляет 17 из 20.

Гоблин2, довольный своим успехом, готовится к следующему ходу.


In [13]:
updated_map = model.update_map(map_info, action=res)
print(updated_map)

{'map': {'grid_size': {'width': 20, 'height': 20}, 'environment': {'terrain': [{'type': 'гора', 'positions': [{'x': 5, 'y': 5}, {'x': 5, 'y': 6}, {'x': 5, 'y': 7}, {'x': 5, 'y': 8}, {'x': 5, 'y': 9}]}, {'type': 'река', 'positions': [{'x': 0, 'y': 10}, {'x': 1, 'y': 10}, {'x': 2, 'y': 10}, {'x': 3, 'y': 10}, {'x': 4, 'y': 10}, {'x': 5, 'y': 10}, {'x': 6, 'y': 10}, {'x': 7, 'y': 10}, {'x': 8, 'y': 10}, {'x': 9, 'y': 10}, {'x': 10, 'y': 10}]}], 'description': 'Поле битвы с рекой посередине карты и горами сверху'}, 'characters': [{'name': 'Герой', 'description': 'Могучий волшебник', 'position': {'x': 2, 'y': 2}, 'inventory': [{'item': 'Посох гадюки', 'damage': '1d8'}], 'spells': [{'spell': 'Огненный шар', 'damage': '26'}], 'stats': {'health': 17, 'max_health': 20, 'armor_class': 10, 'speed': 30}}, {'name': 'Гоблин1', 'description': 'Слабый трусливый гоблин', 'position': {'x': 1, 'y': 18}, 'inventory': [{'item': 'Меч', 'damage': '1d6'}], 'stats': {'health': 7, 'max_health': 7, 'armor_class'

In [9]:
map_info

{'map': {'grid_size': {'width': 20, 'height': 20},
  'environment': {'terrain': [{'type': 'гора',
     'positions': [{'x': 5, 'y': 5},
      {'x': 5, 'y': 6},
      {'x': 5, 'y': 7},
      {'x': 5, 'y': 8},
      {'x': 5, 'y': 9}]},
    {'type': 'река',
     'positions': [{'x': 0, 'y': 10},
      {'x': 1, 'y': 10},
      {'x': 2, 'y': 10},
      {'x': 3, 'y': 10},
      {'x': 4, 'y': 10},
      {'x': 5, 'y': 10},
      {'x': 6, 'y': 10},
      {'x': 7, 'y': 10},
      {'x': 8, 'y': 10},
      {'x': 9, 'y': 10},
      {'x': 10, 'y': 10}]}]},
  'description': 'Поле битвы с рекой посередине карты и горами сверху'},
 'characters': [{'name': 'Герой',
   'description': 'Могучий волшебник',
   'position': {'x': 2, 'y': 2},
   'inventory': [{'item': 'Посох гадюки', 'damage': '1d8'}],
   'spells': [{'spell': 'Огненный шар', 'damage': '26'}],
   'stats': {'health': 20, 'max_health': 20, 'armor_class': 10, 'speed': 30}},
  {'name': 'Гоблин1',
   'description': 'Слабый трусливый гоблин',
   'posit

In [15]:
updated_map

{'map': {'grid_size': {'width': 20, 'height': 20},
  'environment': {'terrain': [{'type': 'гора',
     'positions': [{'x': 5, 'y': 5},
      {'x': 5, 'y': 6},
      {'x': 5, 'y': 7},
      {'x': 5, 'y': 8},
      {'x': 5, 'y': 9}]},
    {'type': 'река',
     'positions': [{'x': 0, 'y': 10},
      {'x': 1, 'y': 10},
      {'x': 2, 'y': 10},
      {'x': 3, 'y': 10},
      {'x': 4, 'y': 10},
      {'x': 5, 'y': 10},
      {'x': 6, 'y': 10},
      {'x': 7, 'y': 10},
      {'x': 8, 'y': 10},
      {'x': 9, 'y': 10},
      {'x': 10, 'y': 10}]}],
   'description': 'Поле битвы с рекой посередине карты и горами сверху'},
  'characters': [{'name': 'Герой',
    'description': 'Могучий волшебник',
    'position': {'x': 2, 'y': 2},
    'inventory': [{'item': 'Посох гадюки', 'damage': '1d8'}],
    'spells': [{'spell': 'Огненный шар', 'damage': '26'}],
    'stats': {'health': 17, 'max_health': 20, 'armor_class': 10, 'speed': 30}},
   {'name': 'Гоблин1',
    'description': 'Слабый трусливый гоблин',
 

#### Тест

In [8]:
with open('test.json', 'r') as file:
    test_data_read = json.load(file)
queries = test_data_read['queries']
answers = test_data_read['answers']

In [9]:
len(queries), len(answers)

(8, 8)

In [10]:
answers

[{'message': 'Гоблин2, находясь на позиции (2, 5), решает атаковать Героя, который находится на позиции (2, 2). Учитывая, что расстояние между ними составляет всего 3 клетки, Гоблин2 может легко добраться до Героя и совершить атаку.\n\nГоблин2 поднимает свою палку и с яростью бросается на врага. Он делает бросок атаки, добавляя свой модификатор ловкости. Бросок кубика на атаку показывает результат 14. С учетом модификатора ловкости (+2), итоговый результат атаки составляет 16.\n\nТеперь Гоблин2 бросает кубик на урон, используя 1d4, и получает 3 урона. Таким образом, Герой получает 3 урона от атаки Гоблина2. \n\nГоблин2 завершает свой ход, оставаясь на позиции (2, 5), с довольной ухмылкой на лице, полагая, что ему удалось хоть немного ослабить могущественного волшебника.',
  'data': {'map': {'grid_size': {'width': 20, 'height': 20},
    'environment': {'terrain': [{'type': 'гора',
       'positions': [{'x': 5, 'y': 5},
        {'x': 5, 'y': 6},
        {'x': 5, 'y': 7},
        {'x': 5,

In [11]:
from agent.process import process_map

In [12]:
#results = []
#for q in queries:
#    result = process_map(q)
#    results.append(result)

In [13]:
#test = {'queries': queries,
#        'answers': results}
#with open("test.json", "w") as fp:
#    json.dump(test , fp, ensure_ascii=False)

In [14]:
from langsmith import Client

client = Client()

# Create a dataset
examples = list(zip(queries, answers))

dataset_name = "Agent RAG DnD"
try:
    dataset = client.create_dataset(dataset_name=dataset_name)
    inputs, outputs = zip(
        *[({"question": text}, {"ground_truth": label}) for text, label in examples]
    )

    client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)
except Exception as e:
    print(e)

Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')


In [15]:
from langsmith.schemas import Example, Run

In [16]:
from langchain import hub
from langchain_openai import ChatOpenAI

# Grade prompt
grade_prompt_answer_helpfulness = prompt = hub.pull("langchain-ai/rag-answer-helpfulness")

def answer_helpfulness_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer helpfulness
    """

    # Get question, ground truth answer, RAG chain answer
    input_question = example.inputs["question"]
    prediction = run.outputs["answer"]

    # LLM grader
    llm = ChatGPTAPI()

    # Structured prompt
    answer_grader = grade_prompt_answer_helpfulness | llm

    # Run evaluator
    score = answer_grader.invoke({"question": input_question,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_helpfulness_score", "score": score}

In [17]:
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response = process_map(example["question"])
    return {"answer": response}

def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = model.call_api(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [ ]:
from langsmith.evaluation import evaluate

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_helpfulness_evaluator],
    experiment_prefix="rag-answer-v-reference",
)

/Users/aydar/Desktop/AI TALENT HUB/DndBattleManager/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rag-answer-v-reference-2e91812e' at:
https://smith.langchain.com/o/ed3ec4a9-24f8-4d1d-ae53-4f7e47fe1b80/datasets/8cbb846d-76bf-4b12-8a21-50ef1761340f/compare?selectedSessions=6ea9ea5c-7a04-48c3-b788-28183c6bd1f9




0it [00:00, ?it/s]